In [764]:
import os
import glob
import json
import math
import re
import
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize

In [765]:
ps = PorterStemmer()

In [766]:
stopword = ['a','an','the','i','is','are','was','were','be','it','has','have','had','do','does','did','and','or','as','of','to']

In [767]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [768]:
def clearIndex():
    with open('docId', 'w', encoding="utf8") as f:
        f.writelines('{}')
    with open('inverted_index', 'w', encoding="utf8") as f:
        f.writelines('{}')

In [769]:
clearIndex()

In [770]:
def create_srt_index(docid):
    """{doc:{id:time, ...}, ...}"""
    srt_index = {}
    with open('srtId', 'r', encoding="utf8") as f:
        srt_index = json.load(f)
        
    with open('docId', 'w', encoding="utf8") as f:
        json.dump(docId, f)
    

In [771]:
with open('docId', 'r', encoding="utf8") as f:
    docId = json.load(f)
with open('inverted_index', 'r', encoding="utf8") as f:
    inverted_lists = json.load(f)

In [772]:
def inverted_index(path, docid):
    global inverted_lists
    docid = int(docid)
    tmp_docs = []
    tmp_lines = []
    #open srt and group by srt list
    with open(path, encoding="utf8") as file:
        for line in file:
            if not line.strip():
                tmp_docs.append(tmp_lines)
                tmp_lines = []
            else: 
                tmp_lines.append(line)


    word_count = 0
    tokenized_docs = []
    rid=0
    doc_count = 0
    for docs in tmp_docs:

        docs = [doc.strip() for doc in docs]
        line = " ".join(docs[2:])
        line = cleanhtml(line)

        # append [line_id, [line_words_list]]
        try:
            rid = int(docs[0])
        except Exception as e:
            print(e)
            rid = doc_count
        tokenized_docs.append([rid,word_tokenize(line)])
        word_count += len(tokenized_docs[-1])
    for line in tokenized_docs:
        for word in line[1]:
            doc_count += 1
            if len(word) > 0 and word not in stopword:
                word = ps.stem(word)
                if word not in inverted_lists:
                    inverted_lists[word] = {docid:[0,0,0]}
                elif not inverted_lists[word].get(docid):
                    inverted_lists[word][docid] = [0,0,0]
                inverted_lists[word][docid][1] += 1.0 / word_count
                inverted_lists[word][docid][2] += 1
                inverted_lists[word][docid].append(line[0])

In [773]:
inverted_lists = {}
docId = {}
folder = glob.glob('srt/*.srt')
with open('docId', 'r', encoding="utf8") as f:
    docId = json.load(f)
with open('inverted_index', 'r', encoding="utf8") as f:
    inverted_lists = json.load(f)
    
counter = len(list(docId.items()))
for file in folder:
    filename = os.path.basename(file)[:-4]
    if (not docId.get(filename)):
        print(file)
        counter += 1
        docId[filename] = counter
        inverted_index(file, counter)
with open('docId', 'w', encoding="utf8") as f:
    json.dump(docId, f)
with open('inverted_index', 'w', encoding="utf8") as f:
    json.dump(inverted_lists, f)

srt\7 Ways to Improve Your NEGOTIATION SKILLS - #7Ways.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Best DIY Robot kit for beginners - Micro_Bit.srt
srt\Build a tower, build a team _ Tom Wujec.srt
srt\Ch. 0 If The World Were A Village.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Conducting Effective Negotiations.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Design Thinking - Tim Brown, CEO and President of IDEO.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Design Thinking In Business.srt
srt\Design Thinking workshop with Justin Ferrell of Stanford d. School at The Irish Times.srt
srt\Designing Your Life Bill Burnett TEDxStanford.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Getting Started with microbit Part 1 Say Hello.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Getting Started with microbit Part 4 Remote Burglar Alarm.srt
invalid literal for int() with base 10: '\ufeff1'
srt\Getting Started with micro_bit Par

In [774]:
def cal_tfidf():
    N = len(docId)
    for word in inverted_lists:
        df = len(inverted_lists[word])
        idf = math.log( N / df, 10 )
        for doc in inverted_lists[word]:
            tf = inverted_lists[word][doc][1]
            inverted_lists[word][doc][0] = tf * idf
    with open('inverted_index', 'w', encoding="utf8") as f:
        json.dump(inverted_lists, f)

In [775]:
cal_tfidf()

In [776]:
def zipper(listA, listB):
    i = 0
    j = 0
    resultDoc = []
    while i < len(listA) and j < len(listB):
        if listA[i] == listB[j]:
            resultDoc.append(listA[i])
            i += 1
            j += 1
        elif listA[i] > listB[j]:
            j += 1
        else:
            i += 1
    return resultDoc

In [777]:
def merge_inverted_lists(queryA, queryB):    
    listA = inverted_lists[queryA]
    listA = list(listA.keys())
    listB = inverted_lists[queryB]
    listB = list(listB.keys())
    
    return zipper(listA, listB)

In [806]:
def query_doc(query_string):
    returnDoc = []

    query_list = word_tokenize(query_string)
    query_list = [ps.stem(word) for word in query_list]
    if len(query_list) >= 1:
        listA = inverted_lists[query_list[0]]
        listA = list(listA.keys())
        returnDoc = listA

    if len(query_list) >= 2:
        listB = inverted_lists[query_list[1]]
        listB = list(listB.keys())
        returnDoc = zipper(listA, listB)

    if len(query_list) >=3:
        for query in query_list[2:]:
            listC = inverted_lists[query]
            listC = list(listC.keys())
            returnDoc = zipper(returnDoc, listC)

    returnDoc_score = []
    for doc in returnDoc:
        returnDoc_score.append([doc,0])
        for query in query_list:
            returnDoc_score[-1][1] += inverted_lists[query][doc][0] 
    returnDoc_score.sort(key= lambda x: x[1], reverse=True)
    return returnDoc_score


In [807]:
query_string  = 'design thinking'
query_doc(query_string)

[[7, 0.02275427221731379],
 [32, 0.02235903708889271],
 [14, 0.02185364084780319],
 [6, 0.02088691427702787],
 [29, 0.015897884475879975],
 [23, 0.012650578770927672],
 [27, 0.012415969383354496],
 [28, 0.010990140427208703],
 [9, 0.01073398789193024],
 [3, 0.00897310595988493],
 [8, 0.00802373181198018],
 [22, 0.007248935373509992],
 [20, 0.0024183612941013864]]

In [808]:
def getPosting(query_string, docId):
    WORD_PER_PASSAGE = 10
    result_docs = []
    result_rank = {}
    result_list_rank = []
    
    doc_count =  0
    for word in word_tokenize(query_string):
        word = ps.stem(word)
        result_docs.append([rid//WORD_PER_PASSAGE for rid in inverted_lists[word][docId][3:]])
    for doc in result_docs:
        doc_count += len(doc)
    for docs in result_docs:
        idf = math.log(doc_count/len(docs))
        for doc in docs:
            if not result_rank.get(doc):
                result_rank[doc] = idf
            else:
                result_rank[doc] += idf
    for item in result_rank:
        result_list_rank.append((item*WORD_PER_PASSAGE, result_rank[item]))
    result_list_rank.sort(key=lambda x: x[1], reverse=True)
    return result_list_rank[0:1]

In [809]:
def get_sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def get_range(srt_path, query):
    with open (srt_path, 'r', encoding="utf8") as f:
        content = f.readlines()
        content = [x.strip() for x in content]
    num ={}
    for i,ele in enumerate(content):
        if ele.isdigit()== True:
            second = get_sec(content[i+1][0:8])
            sub = cleanhtml(content[i+2])
            num[int(ele)] = (second, sub)
    summery = ""
    if num.get(query):
        for i in range(5):
            if num.get(query +1):
                summery = summery + " " + num[query + i][1]
        return (num[query][0], summery)
    else:
        for i in range(5):
            if num.get(1 + i):
                summery = summery + " " + num[query + i][1]
        return (0, summery)


In [810]:
idDoc = list(docId)
def query(query_string):
    res = {}
    for doc in query_doc(query_string):
        docName = idDoc[int(doc[0]) - 1]
        vidName = docName.replace(".transcribed", "") + ".mp4"
        res[vidName] = []
        posting = getPosting(query_string, doc[0])
        for srtId in posting:
            res[vidName].append(get_range('srt/{}.srt'.format(docName), srtId[0] + 1))
    print(res)
query('marshmallow challenge')

{'How to Demonstrate Engineering Principles Science Projects.mp4': [(0, ' When you think of natural disasters, there volcanoes, tornadoes, hurricanes. But, if and tell you that we’re about to do an experiment gonna be, like, “Wait, what? Marshmallows')], 'Group Initiative that Inspires Powerful Metaphors - Marshmallow Challenge.mp4': [(71, ' to work together is to combine all of those resources to create not only a self-supporting structure but most importantly you may have the tallest structure and still not')], 'Build a tower, build a team _ Tom Wujec.mp4': [(16, " Several years ago here at TED, Peter Skillman introduced a design challenge called the marshmallow challenge. And the idea's pretty simple: Teams of four have to build the tallest free-standing structure")], 'Peter Skillman Marshmallow Design Challenge.mp4': [(0, " on stage now for three minutes Peter Skillman thanks here's what you get a team of for 18 minutes 20 pieces of spaghetti a meter of tape and a piece")]}


In [783]:
tmp_docs = []
tmp_lines = []
with open('srt/'+list(docId)[25] + '.srt', encoding="utf8") as file:
    for line in file:
        if not line.strip():
            tmp_docs.append(tmp_lines)
            tmp_lines = []
        else: 
            tmp_lines.append(line)

for rec in tmp_docs:
    print(rec)
# recList = [2,3,8,9,33,39,65,70,72,129,134,137,101]
# for rec in recList:
#     print(tmp_docs[rec-1])
# for line in tmp_docs:
#     print(line)

['\ufeff1\n', '00:00:00,060 --> 00:00:06,899\n', 'this is the<font color="#CCCCCC"> bass you wonder hi never to</font>\n']
['2\n', '00:00:03,179 --> 00:00:13,830\n', 'the<font color="#CCCCCC"> bass</font><font color="#E5E5E5"> because</font><font color="#CCCCCC"> I was awaiting</font><font color="#E5E5E5"> realer</font>\n']
['3\n', '00:00:06,899 --> 00:00:17,580\n', '<font color="#E5E5E5">it\'s amazing no</font><font color="#CCCCCC"> so we had to start</font><font color="#E5E5E5"> with</font>\n']
['4\n', '00:00:13,830 --> 00:00:20,640\n', 'a<font color="#E5E5E5"> sturdy base we decided we wanted</font><font color="#CCCCCC"> to do</font>\n']
['5\n', '00:00:17,580 --> 00:00:23,930\n', 'criss cross<font color="#E5E5E5"> section to give it more</font>\n']
['6\n', '00:00:20,640 --> 00:00:26,609\n', 'stability so<font color="#CCCCCC"> that</font><font color="#E5E5E5"> it could be taller</font>\n']
['7\n', '00:00:23,930 --> 00:00:30,240\n', 'gradually got swimmers we went up so<font color="#E

In [784]:
for i in ['key: {}, name: "{}", pos:[0]'.format(key, v) for key,v in enumerate(os.listdir('video/'))]:
    print('{' + i + '},')

{key: 0, name: "7 Ways to Improve Your NEGOTIATION SKILLS - #7Ways.mp4", pos:[0]},
{key: 1, name: "Best DIY Robot kit for beginners - Micro_Bit.mp4", pos:[0]},
{key: 2, name: "Build a tower, build a team _ Tom Wujec.mp4", pos:[0]},
{key: 3, name: "Ch. 0 If The World Were A Village.mp4", pos:[0]},
{key: 4, name: "Conducting Effective Negotiations.mp4", pos:[0]},
{key: 5, name: "Design Thinking - Tim Brown, CEO and President of IDEO.mp4", pos:[0]},
{key: 6, name: "Design Thinking In Business.mp4", pos:[0]},
{key: 7, name: "Design Thinking workshop with Justin Ferrell of Stanford d. School at The Irish Times.mp4", pos:[0]},
{key: 8, name: "Designing Your Life Bill Burnett TEDxStanford.mp4", pos:[0]},
{key: 9, name: "Getting Started with microbit Part 1 Say Hello.mp4", pos:[0]},
{key: 10, name: "Getting Started with microbit Part 4 Remote Burglar Alarm.mp4", pos:[0]},
{key: 11, name: "Getting Started with micro_bit Part 2_ Electronic Magic 8 Ball.mp4", pos:[0]},
{key: 12, name: "Group Init

In [785]:
docId

{'7 Ways to Improve Your NEGOTIATION SKILLS - #7Ways': 1,
 'Best DIY Robot kit for beginners - Micro_Bit': 2,
 'Build a tower, build a team _ Tom Wujec': 3,
 'Ch. 0 If The World Were A Village': 4,
 'Conducting Effective Negotiations': 5,
 'Design Thinking - Tim Brown, CEO and President of IDEO': 6,
 'Design Thinking In Business': 7,
 'Design Thinking workshop with Justin Ferrell of Stanford d. School at The Irish Times': 8,
 'Designing Your Life Bill Burnett TEDxStanford': 9,
 'Getting Started with microbit Part 1 Say Hello': 10,
 'Getting Started with microbit Part 4 Remote Burglar Alarm': 11,
 'Getting Started with micro_bit Part 2_ Electronic Magic 8 Ball': 12,
 'Group Initiative that Inspires Powerful Metaphors - Marshmallow Challenge': 13,
 'How It Works Design Thinking': 14,
 'How to Demonstrate Engineering Principles Science Projects': 15,
 'If The World Was 100 People by Jay Shetty': 16,
 'If the World Was Only 100 People': 17,
 "If the World Were a Village of 100 People A Sto

In [724]:
''' rename sub to be same as video '''

srt = os.listdir('srt/')
for filename in os.listdir("video/"):
    for file in srt:
        if filename[:-5] in file:
            os.rename('srt/' + file, 'srt/{}.srt'.format(filename[:-4]))

In [786]:
inverted_lists

{'what': {1: [0.004887390995593717,
   0.0684931506849315,
   60,
   0,
   51,
   52,
   52,
   58,
   66,
   67,
   69,
   70,
   82,
   92,
   95,
   98,
   99,
   106,
   107,
   108,
   113,
   133,
   138,
   139,
   142,
   143,
   144,
   145,
   145,
   150,
   151,
   153,
   165,
   183,
   190,
   206,
   215,
   216,
   217,
   218,
   220,
   237,
   240,
   254,
   260,
   278,
   285,
   294,
   295,
   298,
   306,
   308,
   323,
   332,
   333,
   341,
   345,
   348,
   375,
   395,
   407,
   410,
   433],
  2: [0.0010493515961127688, 0.014705882352941176, 3, 20, 72, 83],
  3: [0.0028454810152260357,
   0.03987730061349694,
   13,
   26,
   37,
   56,
   65,
   66,
   69,
   77,
   77,
   112,
   113,
   114,
   121,
   134],
  4: [0.0005405750646641535, 0.007575757575757576, 2, 128, 130],
  5: [0.0024837444595446566,
   0.03480783176214653,
   96,
   10,
   35,
   88,
   93,
   98,
   110,
   169,
   170,
   175,
   205,
   209,
   209,
   211,
   213,
   233,
   2